In [97]:
# import libraries for data manipulation
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split

# vectorizer for dictionaries
from sklearn.feature_extraction import DictVectorizer

# decision tree regressor model
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text

# random forest regressor model 
from sklearn.ensemble import RandomForestRegressor

In [1]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import xgboost as xbg

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/ciarawilliams/Desktop/ml_zoomcamp/ml_zoomcamp/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <E8D72161-CCD1-3423-9388-36D4CA0A7524> /Users/ciarawilliams/Desktop/ml_zoomcamp/ml_zoomcamp/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptex

In [60]:
data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
!wget $data

--2025-11-08 17:03:00--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘car_fuel_efficiency.csv’

car_fuel_efficiency 100%[===================>] 853.70K  --.-KB/s    in 0.07s   

2025-11-08 17:03:00 (11.3 MB/s) - ‘car_fuel_efficiency.csv’ saved [874188/874188]



In [61]:
# read in the data and save in a DataFrame
df = pd.read_csv("car_fuel_efficiency.csv")

# Data Preparation

First, let's make the names lowercase:

```python
df.columns = df.columns.str.lower().str.replace(' ', '_')
```

Preparation:

* Remove the `student_id` column.
* Fill missing values with zeros.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

In [62]:
df_copy = df.copy()

df_copy.columns = df_copy.columns.str.lower().str.replace(' ', '_')

In [63]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB


In [64]:
# get count of missing values per feature
df_copy.isna().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [65]:
# separate continuous and categorical features
num_cols = list(df_copy.columns[df_copy.dtypes != 'object'])
cat_cols = list(df_copy.columns[df_copy.dtypes == 'object'])

## Missing Value Handling
* Check if the missing values are presented in the features.
* If there are missing values:
    * For caterogiral features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [66]:
# handle missing values for continuous features
for n in df_copy[num_cols]:
    df_copy[n] = df_copy[n].fillna(0)

## Split the data
* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `1`.
* Make sure that the target value `y` is not in your dataframe.

In [67]:
df_full_train, df_test = train_test_split(df_copy, test_size=0.2, random_state=1)

df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [68]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_train = df_train['fuel_efficiency_mpg'].values
y_test = df_test['fuel_efficiency_mpg'].values
y_val = df_val['fuel_efficiency_mpg'].values

df_train.drop(['fuel_efficiency_mpg'], axis=1, inplace=True)
df_test.drop(['fuel_efficiency_mpg'], axis=1, inplace=True)
df_val.drop(['fuel_efficiency_mpg'], axis=1, inplace=True)

# Decision Tree Regressor

In [69]:
def decision_tree_reg(df_train, y_train, max_depth=None):
    train_dict = df_train.to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)

    model = DecisionTreeRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)

    return (dv, model)

## Question 1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?
* vehicle_weight

In [70]:
# max_depth = 1
dv_1, dt_1 = decision_tree_reg(df_train=df_train, y_train=y_train, max_depth=1)

print(export_text(dt_1, feature_names=list(dv_1.get_feature_names_out())))


|--- vehicle_weight <= 3022.11
|   |--- value: [16.88]
|--- vehicle_weight >  3022.11
|   |--- value: [12.94]



# Random Forest Regressor

In [75]:
def rmse(y, y_pred):
	'''Calculate Root Mean Squared Error for Model Validation.'''
	error = y - y_pred
	sqr_error = error ** 2
	mse = sqr_error.mean()
	
	return np.sqrt(mse)

In [89]:
def rf_reg(df_train, df_val, y_train, y_val, n_est=10, max_depth=None):
    train_dict = df_train.to_dict(orient='records')
    val_dict = df_val.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    dv.fit(val_dict)
    X_val = dv.transform(val_dict)

    rf = RandomForestRegressor(max_depth=max_depth, n_estimators=n_est, random_state=1, n_jobs=1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)

    rf_rmse = rmse(y_val, y_pred)

    return [dv, rf, rf_rmse]

## Question 2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on the validation data?

* 0.45

In [90]:
rf_rmse = rf_reg(df_train, df_val, y_train, y_val)[2]

rf_rmse

np.float64(0.4570712602619733)

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.
- 150,  (150: 0.442)

If it doesn't stop improving, use the latest iteration number in
your answer.

In [79]:
rmses = {}
n_est = np.arange(10, 200, 10)

for n in n_est:
    n_rmse = rf_reg(df_train, df_val, y_train, y_val, n_est=n)[2]
    rmses[n] = round(n_rmse, 3)

rmses

{np.int64(10): np.float64(0.457),
 np.int64(20): np.float64(0.452),
 np.int64(30): np.float64(0.45),
 np.int64(40): np.float64(0.448),
 np.int64(50): np.float64(0.446),
 np.int64(60): np.float64(0.445),
 np.int64(70): np.float64(0.445),
 np.int64(80): np.float64(0.445),
 np.int64(90): np.float64(0.444),
 np.int64(100): np.float64(0.444),
 np.int64(110): np.float64(0.443),
 np.int64(120): np.float64(0.443),
 np.int64(130): np.float64(0.443),
 np.int64(140): np.float64(0.443),
 np.int64(150): np.float64(0.442),
 np.int64(160): np.float64(0.442),
 np.int64(170): np.float64(0.442),
 np.int64(180): np.float64(0.442),
 np.int64(190): np.float64(0.442)}

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

* 10, (10: 0.442)

In [80]:
mean_rmses = {}
n_est = np.arange(10, 200, 10)
max_depths = [10, 15, 20, 25]

for m in max_depths:
    rmses = 0
    for n in n_est:
        n_rmse = rf_reg(df_train, df_val, y_train, y_val, n_est=n, max_depth=m)[2]
        rmses += n_rmse

    m_rmse = rmses/len(n_est)
    
    mean_rmses[m] = round(m_rmse, 3)

mean_rmses

{10: np.float64(0.442),
 15: np.float64(0.445),
 20: np.float64(0.446),
 25: np.float64(0.446)}

# Question 5

We can extract feature importance information from tree-based models. 
For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `vehicle_weight`

In [94]:
dv, rf_1 = rf_reg(df_train, df_val, y_train, y_val, max_depth=20)[:2]

# get the feature importances from the Random Forest
rf_importances = rf_1.feature_importances_
# get the feature labels
rf_cols = list(dv_1.get_feature_names_out())
# create a dataframe of row=feature_name col=Importance 
rf_importance_df = pd.DataFrame(rf_importances, index=rf_cols, columns=['Importance']).sort_values(by='Importance', ascending=False)

rf_importance_df


,Importance
vehicle_weight,0.959176
horsepower,0.016042
acceleration,0.011454
engine_displacement,0.003282
model_year,0.003165
num_cylinders,0.002331
num_doors,0.001628
origin=USA,0.000565
origin=Europe,0.000494
origin=Asia,0.000485


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds: